# find tracerRNA

03/12/2019: migliorare e vedere se si riesce a prendere in considerazione le cose giuste. 
of FLJLKKHA_00012
problem: ha 2 active working cas loci. e il tracr mi sa che l'ha trovato in tutti e due, ma solo uno è segnato.
l'altro (greppato a mano) è:
TODO: qualcosa di sbagliato accade quando checcki la posizione di cas9 nel genoma, traduce male il primo aminoacido , e solo il primo parrebbe ... hm

- Cas9 : NODE_49_length_176785_cov_12.2725       Prodigal:2.6    CDS     15614   18553   .       -       0       ID=FLJLKKHA_00012;Parent=FLJLKKHA_00012_gene;eC_number=3.1.-.-;Name=cas9_1;gene=cas9_1;inference=ab initio prediction:Prodigal:2.6,similar to AA sequence:UniProtKB:Q9CLT2;locus_tag=FLJLKKHA_00012;product=CRISPR-associated endonuclease Cas9;protein_id=gnl|X|FLJLKKHA_00012
- Cas1 : NODE_49_length_176785_cov_12.2725       Prodigal:2.6    CDS     14715   15614   .       -       0       ID=FLJLKKHA_00011;Parent=FLJLKKHA_00011_gene;eC_number=3.1.-.-;Name=cas1_1;gene=cas1_1;inference=ab initio prediction:Prodigal:2.6,similar to AA sequence:UniProtKB:G3ECR2;locus_tag=FLJLKKHA_00011;product=CRISPR-associated endonuclease Cas1;protein_id=gnl|X|FLJLKKHA_00011
- Cas2 : NODE_49_length_176785_cov_12.2725       Prodigal:2.6    CDS     14378   14707   .       -       0       ID=FLJLKKHA_00010;Parent=FLJLKKHA_00010_gene;eC_number=3.1.-.-;Name=cas2_1;gene=cas2_1;inference=ab initio prediction:Prodigal:2.6,similar to AA sequence:UniProtKB:G3ECR3;locus_tag=FLJLKKHA_00010;product=CRISPR-associated endoribonuclease Cas2;protein_id=gnl|X|FLJLKKHA_00010


In [1]:
# First created Thu Jul 11 09:51:15 CEST 2019
# Made by L-F-S
# At the University Of Trento, Italy

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein, RNAAlphabet
from Bio import pairwise2


sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/utils/')
import filename_discrepancies
sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/9casminer_v2/')
import locus


feature="Cas9" #WARNING!!! CHANGE THIS!!
#outpath="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/out/"+feature+"/"
datadir="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"




In [2]:
# Import data
cas_dataset=pd.read_csv("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/known_"+feature+"prokkacas9_variants_table.csv", index_col=0)
cas_dataset.columns

Index(['Seq ID', 'Seq Description', 'Seq', 'Contig', 'Genome Name', 'Study',
       'Sample Name', 'SGB ID', 'Pos', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_cas1', 'prokka_cas2', 'uSGB', 'Level of estimated taxonomy',
       'Estimated taxonomy', 'prokka_cas9'],
      dtype='object')

## inspect casdataset, especially the last column..

In [8]:
seqid ="AOPFDEBE_00006"  #00012
print("SEQ ID: ", seqid)
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0])
print(len(cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]))
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])
print("----------------------------------------------------------------------")


SEQ ID:  AOPFDEBE_00006
NODE_411_length_19879_cov_2.8009
Obregon-TitoAJ_2015
Obregon-TitoAJ_2015__SM31__bin.19
17319
978
[('.', '10862', '12283')]
----------------------------------------------------------------------


## double cas check: check how many cas9s are there in that genome


In [9]:
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
cas_dataset[cas_dataset["Genome Name"]==genomename]

,Seq ID,Seq Description,Seq,Contig,Genome Name,Study,Sample Name,SGB ID,Pos,pilercr_CRISPR,minced_CRISPR,prokka_cas1,prokka_cas2,uSGB,Level of estimated taxonomy,Estimated taxonomy,prokka_cas9
19055,AOPFDEBE_00006,AOPFDEBE_00006 CRISPR-associated endonuclease ...,MSLIFGFDLGIASIGWAAVKMSDPTASDTDAAAPDNRPVGEILGAG...,NODE_411_length_19879_cov_2.8009,Obregon-TitoAJ_2015__SM31__bin.19,Obregon-TitoAJ_2015,SM31,17319,6519 9455,"[('ID=CRISPR1', '11227', '6')]","[('.', '10862', '12283')]","[('ID=AOPFDEBE_00007', '9510', '10424')]","[('ID=AOPFDEBE_00008', '10453', '10758')]",Yes,Other,-,"('ID=AOPFDEBE_00006', '6519', '9455')"


In [17]:
spacers=[]
repeats=[]
repeat_start_pos=[]


cas9_aa=cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]
dataset=cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0]
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
SGB=cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0]
contigname=cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0]
samplename = genomename.split("__")[1]

print("\n-----------------------------------------------------\n"+feature+" id:\t"+seqid+"\nSGB:\t"+str(SGB)+"\nGenome Name:\t",\
      genomename+"\nContig:\t"+cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0],"\n-----------------------------------------------------\n")


########################################################################

# get CRISPR spacer and repeat sequence

cr=locus.CRISPRarray(feature=feature, contigname=contigname, genomename=genomename, datasetname=dataset)
cr.get_CRISPR_array()
spacers=cr.spacers
repeats=cr.repeats
repeat_start_pos=cr.repstartpos

# get whole locus sequence todo this could become a function of its own, dentro utils, gli dai seqid della cas e ti dà tt il locus

# > get start and stop position of all Cass. questo devi metterlo nella tabella di cas non ci sono santi.
cas_position={feature:[int(n) for n in cas_dataset[cas_dataset["Seq ID"]==seqid]["Pos"].iloc[0].split()],\
              "Cas1":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["prokka_cas1"].iloc[0])[0][1:]],\
              "Cas2":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["prokka_cas2"].iloc[0])[0][1:]],\
              "CRISPR":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])[0][1:]]
             }
print("Locus features and positions: ", cas_position)
# so già che all casses inside here are on the same contig which is 'contig', xke le ho estratte per essere così
# FALSO! è cosi in cas_dataset, non in tabellazzza! mi ero confuso! trovato il bug!

genomefilename= "/shares/CIBIO-Storage/CM/scratch/tmp_projects/epasolli_darkmatter/allcontigs/ALLreconstructedgenomes/"+str(SGB)+"/"+genomename+".fa"
print("+"*80)
print(cas_position)
print("Retrieving locus")


# access contig:

for record in SeqIO.parse(genomefilename,"fasta"):  #c'è modo di non fare questo 'ciclo'?
    if record.id.startswith(contigname):
        #test se effettivamente la seq di cas è dove voglio io

        SeqIO.write(record, "/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb", "fasta")
        print("CRISPR si trova", cas_position["CRISPR"])
        print("Cas9 si trova", cas_position[feature])
        print("Cas2 si trova", cas_position["Cas2"])
        print("Cas1 si trova", cas_position["Cas1"])

        #estrai la posizione di cas9 dal contig, e traducila
        temp_alt_cas9start=98503
        temp_alt_cas9stop=101700
        cas9start=cas_position[feature][0]
        cas9stop=cas_position[feature][1]
        cas9_nnseq=record.seq[cas9start-1:cas9stop-3] #gff should have a 1-based positional annotation (ma sto andando a occhio finchè non sono uguali, è già un oggetto Bio.Seq

        my_translated_cas9=cas9_nnseq.transcribe().translate()

  
        print(len(cas9_nnseq)/3,len(cas9_aa))


        if my_translated_cas9==cas9_aa:
            print("Cas locus on plus strand")
            plus=True
        if not my_translated_cas9==cas9_aa:
            #proviamo col reverse complement
            cas9_nnseq=record.seq[cas9start+2:cas9stop]
            my_translated_cas9=cas9_nnseq.reverse_complement().transcribe().translate()
            print("Cas locus on minus strand")
        print("Is Cas9 translated from the genome equal to the orignal annotation?", my_translated_cas9==cas9_aa)
        print(my_translated_cas9)
        print("+"*80)
        print(cas9_aa)
        
        alignments = pairwise2.align.localxx(my_translated_cas9,cas9_aa)
        print(pairwise2.format_alignment(*alignments[0]))

        #opzione 1 blast
      #1. salva np.uniq(repeats) in un fasta file: build temporary query file for blastn search

        print("let's now blast the repeat against  the genome")
        blast_folder="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"
        os.chdir(blast_folder)
        tempfile=open("temp_repeat_seq", "w")
        tempfile.close()
        tempfile=open("temp_repeat_seq", "a")
        for n, repeat in enumerate([Seq(sequence) for sequence in np.unique(repeats)]):
            tempfile.write(">rpt"+str(n+1)+"|"+contigname+"|"+genomename+"|"+seqid+"\n"+str(repeat)+"\n") 
        tempfile.close()

        #2. Blast query file against db
        print("Running blastn of query file agianst all contigs")

        #3. make db file
        dbfile="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb"
        os.system("makeblastdb -in "+dbfile+" -parse_seqids  -dbtype nucl")



        blastoutfile="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"\
        +str(SGB)+"__"+seqid+"__"+genomename+"__"+feature+".trcr.blastout"
        blastn_command = "blastn -out "+blastoutfile+" -outfmt \"6  qseqid sseqid pident qlen length mismatch gapopen qseq sseq sstart send evalue sstrand\" -query temp_repeat_seq -db "+dbfile+" -evalue 0.001 -word_size 11 -penalty -2"
        print(blastn_command)
        os.system(blastn_command)
       # os.system("rm /shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracerRNA/temp*")



-----------------------------------------------------
Cas9 id:	AOPFDEBE_00006
SGB:	17319
Genome Name:	 Obregon-TitoAJ_2015__SM31__bin.19
Contig:	NODE_411_length_19879_cov_2.8009 
-----------------------------------------------------



CRISPR 1   Range: 10862 - 12283

POSITION	REPEAT				SPACER

--------	------------------------------------	------------------------------

10862		ATTATACCAAAGACGATTTGAGAAGCAACCTATAAC	AAGACGCACTCAACAGCACCAATATGCAGG	[ 36, 30 ]

10928		ATTATACCAAAGACGATTTGAGAAGCAACCTATAAC	AACAAGTATACGCAATGTTTAACCAAATCA	[ 36, 30 ]

10994		ATTATACCAAAGACGATTTGAGAAGCAACCTATAAC	AGGTCAGGATTATCAACCACGTATCGATTC	[ 36, 30 ]

11060		ATTATACCAAAGACGATTTGAGAAGCAACCTATAAC	CTATACTAAGGGCATATCAATAAGGACATA	[ 36, 30 ]

11126		ATTATACCAAAGACGATTTGAGAAGCAACCTATAAC	ATGTATATTTCATTGTTGATATTTTCTTTG	[ 36, 30 ]

11192		ATTATACCAAAGACGATTTGAGAAGCAACCTATAAC	CCAGCCATGTTATATATTTGTGCCGCTTGT	[ 36, 30 ]

11258		ATTATACCAAAGACGATTTGAGAAGCAACCTATAAC	TGTAGTTGCTGTCGTAATACCCGACCGTCA	[ 36, 30 ]

11324		ATTATACCA

## write Cas9 protein aa sequence

In [13]:
f=open("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/to_give_to_sixnis/"+seqid+".faa","w")
f.write(">"+seqid+"\n"+cas9_aa)
f.close()

## 4 parse blast output

In [20]:
#4. parse blast output: only print output if it is NOT one of the old repeats.
print("\n-----------------------------------------------------\nParsing BLAST output, looking for hit which are NOT the same repeats\ni.e. not starting with:", repeat_start_pos,":\n-----------------------------------------------------\n")
f=open(blastoutfile,"r")
for line in f.readlines():
    start_pos=line.split("\t")[9]
    if start_pos not in repeat_start_pos:
        if str(int(start_pos)+1) not in repeat_start_pos:
            if str(int(start_pos)-1) not in repeat_start_pos:
                print(line) 


-----------------------------------------------------
Parsing BLAST output, looking for hit which are NOT the same repeats
i.e. not starting with: ['10862', '10928', '10994', '11060', '11126', '11192', '11258', '11324', '11390', '11456', '11522', '11588', '11654', '11720', '11786', '11852', '11918', '11984', '12050', '12116', '12182', '12248'] :
-----------------------------------------------------



## extract sequences

In [22]:
# attenzioe: strano: poiché è sul minus strand, quindi start e stop sono al contrario. ma secondo i miei calcoli, per come è fatto blast, dovrebbe hittare sul plus strand..
#qst vuol dire che sul - c'è la repeat,ee onn lantirepeat. quindi funge male....
# che è testimoniato piu a valle dal fatto che trovo n  terminatore la  vicino, ma lo trovo dopo...
#omg che sia un tracrRNA NON trans?
# D:

# questo potrebbe voler dire che il crispr arraay è in realtà sullo strand opposto?

matched_rep_start=98407#18665
matched_rep_stop=98430#18691

In [23]:
# this is the sequence matching the repeat:
matched_rep=Seq("ATTTAGCAGATGATTATTTTGTGT")#"TTATCGCTTCCTTTCCAATTAAAACGC")

#rpts which match:
rpt2=Seq("ATTATAGCAGATGTTTATTTTGTGTCAAACCGCAAC")#"GTTATCGCTTCCTTTCCAGCCTAAACGCGGTATAAT")

# since repeat is on the + strand, and this is all on the + strand, tracrRNA should be its reverse complement:
antirepeat=matched_rep.reverse_complement()
#this should go frombo to bo  = x nt  (x+1)
print(antirepeat)
len(antirepeat)

ACACAAAATAATCATCTGCTAAAT


24

In [28]:
# print terminators

f=open(blast_folder+str(SGB)+"__"+seqid+"__"+genomename+".terminators")
for line in f.readlines():
    if line.strip().startswith("98"): #18  <--------------------------------------------------
        print(line)
f.close()

   9877      Both + TCGTTCAACAGCCCGCAGACGCGTTTGACGTCCGCGGGTTTTCCTTTGATA NA

  98331     Erpin - ACACAAAATTAAGGGGGCTGACTGCGCGTCGCCCCCTTTTCATTTAAAAA NA

  98334     Erpin + TTAAATGAAAAGGGGGCGACGCGCAGTCAGCCCCCTTAATTTTGTGTG NA

  98428      Both + AATGATAGAAACATCTTGCCACATCTATCCTAATAGTGCAAGATATTTTTAGTTGATT NA



In [7]:
# this is the rho-independent trnascription terminator,
# going from 14719 or 14722 to bo (+1 perché conti il primo tipo), 

terminator_end=18895 -1 # qui è sbagliato, mavabbe proviamo
terminator=Seq("ATACGGAAAAAGCCTTGAAGCGGCGGACTTCAAGGCTTTAAGAATTGGT".upper())
print("terminator length: ", len(terminator))
terminator_start=terminator_end+len(terminator) # occhio che qui start e stop si riferisce al rev.compl,
                                                # mentre nel antirepeat si riferisce alla direzione base


terminator length:  49


In [29]:
#se terminator on + strand:
terminator_start=98428#18895-1
terminator=Seq("ATACGGAAAAAGCCTTGAAGCGGCGGACTTCAAGGCTTTAAGAATTGGT".upper())
print("terminator length: ", len(terminator))
terminator_end=terminator_start+len(terminator)

terminator length:  49


In [31]:
# Open genome and extract the whole sequence
# find this inside the genome, and link it to the thing found by ARNold:
#solo per lui:
contigname="NODE_49_length_176785_cov_12.2725"
for record in SeqIO.parse("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/selected_loci/"+seqid+"/"+genomename+".fa", "fasta"):
    if record.id==contigname:
        contigseq=record.seq
print(len(contigseq[matched_rep_start-1:matched_rep_stop]))
#check antirepeat
print("test repeat: ", antirepeat, len(antirepeat), contigseq[matched_rep_start-1:matched_rep_stop]==antirepeat) # -1 why? Indexing diverso evidentemente col blastout, vbb
# check terminator
print("test terminator: " ,terminator, len(terminator), contigseq[terminator_start:terminator_end]==terminator)


pre_tracrRNA=contigseq[matched_rep_start:terminator_end]
print("pre tracrRNA:", pre_tracrRNA, len(pre_tracrRNA))

# add mario
mario=contigseq[matched_rep_stop:terminator_start]
print("mario sequence:",  mario, len(mario))  # ho fatto i check, il -1 sopra è giusto

24
test repeat:  ACACAAAATAATCATCTGCTAAAT 24 False
test terminator:  ATACGGAAAAAGCCTTGAAGCGGCGGACTTCAAGGCTTTAAGAATTGGT 49 False
pre tracrRNA: AAGCGACGATGGTTCCTTTGACGACGGAACCTTCCAATCCTTTGTCTTCACCGAACATTTCGTTGAACAG 70
mario sequence:  0


In [ ]:
#mario un po' lungo, ma a me mi sembra che al primo TTTTTATCG ci si possa fermare

In [32]:
print(contigseq[matched_rep_start-100:terminator_end+100])

TGATCGCCGACTTTCATTTCGGCGTTAAAGCCGAATTCACGGCGCGCGATGCGACCTTCGGATTTCAAACCGACGTCGATTGTCGCAAAATCGTCGGACAAAGCGACGATGGTTCCTTTGACGACGGAACCTTCCAATCCTTTGTCTTCACCGAACATTTCGTTGAACAGTTCGGCAAAGTTTTCTTTCATAGCCGGGATTTCGGCGGCTGTATTTTTTGTTGCCATTAAAAAACTTCCTTTAAGACTTTACTTGCCATTCGGTTGTATC


In [33]:
# Now, align with the rest of the stuff to see what's up
# first up, with the repeat, lets take the rpt3, which has only got 1 mismatch

alignments = pairwise2.align.localxx(rpt2, antirepeat.reverse_complement())
print(pairwise2.format_alignment(*alignments[0]))
print("rpt length:", len(rpt2))


ATTATAGCAGATGT-TTATTTTGTGTCAAACCGCAAC
||| |||||||||  |||||||||||
ATT-TAGCAGATG-ATTATTTTGTGT-----------
  Score=23

rpt length: 36


In [34]:
# align antirepat + mario with repeat
alignments = pairwise2.align.globalxx( rpt2,(antirepeat+mario[:4]).reverse_complement())
print(pairwise2.format_alignment(*alignments[0]))

ATTATAGCAGATGT-TTATTTTGTGTCAAACCGCAAC
||| |||||||||  |||||||||||           
ATT-TAGCAGATG-ATTATTTTGTGT-----------
  Score=23



In [35]:
#no non fa nulla quel CTTT in più

In [36]:
# I would probably say that the tracr is just, simply la parte di pre_tracrRNA la cui antirepeat si lega alla repeat, 
# che, in questo caso, è tutta.
tracrRNA = pre_tracrRNA

print("\n----------------------------------------------------------------------")
print("         tracrRNA sequence for", feature,"protein id", seqid)
print("----------------------------------------------------------------------\n")
print("- contig name:", cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print("- Dataset: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print("- Bin: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print("- SGB: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0])
print("----------------------------------------------------------------------\n")
print("- Length: ", len(tracrRNA),"\n- position: ", terminator_end,matched_rep_stop)
print(tracrRNA)


----------------------------------------------------------------------
         tracrRNA sequence for Cas9 protein id FLJLKKHA_00273
----------------------------------------------------------------------

- contig name: NODE_56_length_167430_cov_15.277
- Dataset:  BritoIL_2016
- Bin:  BritoIL_2016__M1.29.ST__bin.30
- SGB:  9311
----------------------------------------------------------------------

- Length:  70 
- position:  98477 98430
AAGCGACGATGGTTCCTTTGACGACGGAACCTTCCAATCCTTTGTCTTCACCGAACATTTCGTTGAACAG


# Print output


In [ ]:
# write file to write output:
strand=-1
n_bases_after_TTTT=4  #to check
#

f=open(datadir+"8outputv1/"+seqid+"/"+seqid+"inputforoutput","w")
f.write(str(matched_rep)+"\n")
f.write(str(rpt3)+"\n")    #check
f.write(str(matched_rep_start)+"\n")
f.write(str(matched_rep_stop)+"\n")
f.write(str(strand)+"\n")  #strand
f.write(str(14653-1-(strand*n_bases_after_TTTT))+"\n")  #terminator end from ARNold # -1, for indexing reasons. This position is the stop if strand = -1, the start, if strand = +1
f.write(str(terminator[:-n_bases_after_TTTT])+"\n")  # to change if n_bases_after_TTTT 
f.write(feature+"\n")
f.write(str(Cas9_strand))
f.close()